In [9]:
from pathlib import Path
import os

yr = 2024
d = 17

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

# inp_path = 'C:\\Users\\riggi\\Projects\\Advent_Of_Code\\Input\\2024\\17_ex1.txt'

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [10]:
import copy

def format_input(inp):
  import re

  registers = {'A': int(re.sub('[^0-9]','', inp[0])),
               'B': int(re.sub('[^0-9]','', inp[1])),
               'C': int(re.sub('[^0-9]','', inp[2]))}
  
  program = [int(x) for x in re.sub('[^0-9,]','', inp[4]).split(',')]
  return registers, program

In [11]:
formatted_input = format_input(inp)

In [12]:
formatted_input

({'A': 28422061, 'B': 0, 'C': 0},
 [2, 4, 1, 1, 7, 5, 1, 5, 4, 2, 5, 5, 0, 3, 3, 0])

In [13]:
def adv(opcode, operand, combo, pointer, registers, out):
  '''0'''
  assert(opcode==0)
  registers['A'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out

def bxl(opcode, operand, combo, pointer, registers, out):
  '''1'''
  assert(opcode==1)
  registers['B'] = registers['B'] ^ operand
  pointer += 2
  return pointer, registers, out

def bst(opcode, operand, combo, pointer, registers, out):
  '''2'''
  assert(opcode==2)
  registers['B'] = combo%8
  pointer += 2
  return pointer, registers, out

def jnz(opcode, operand, combo, pointer, registers, out):
  '''3'''
  assert(opcode==3)
  if registers['A']!=0:
    pointer = operand-2
  pointer += 2
  return pointer, registers, out

def bxc(opcode, operand, combo, pointer, registers, out):
  '''4'''
  assert(opcode==4)
  registers['B'] = registers['B'] ^ registers['C']
  pointer += 2
  return pointer, registers, out

def out(opcode, operand, combo, pointer, registers, out):
  '''5'''
  assert(opcode==5)
  out.append(combo%8)
  pointer += 2
  return pointer, registers, out


def bdv(opcode, operand, combo, pointer, registers, out):
  '''6'''
  assert(opcode==6)
  registers['B'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out


def cdv(opcode, operand, combo, pointer, registers, out):
  '''7'''
  assert(opcode==7)
  registers['C'] = int(registers['A']/(2**combo))
  pointer += 2
  return pointer, registers, out

  
opcode2func = {0:adv,
               1:bxl,
               2:bst,
               3:jnz,
               4:bxc,
               5:out,
               6:bdv,
               7:cdv}


def execute(registers, program):
  pointer = 0
  out = []
  history = [(copy.deepcopy(pointer), copy.deepcopy(registers), copy.deepcopy(out))]
  while pointer < len(program):
    operand_to_combo = {
                      0:0,
                      1:1,
                      2:2,
                      3:3,
                      4: registers['A'],
                      5: registers['B'],
                      6: registers['C']
                      }
    # print('============================================')
    # print("-----------------BEFORE---------------------")
    opcode = program[pointer]
    operand = program[pointer+1]
    combo = operand_to_combo[operand]
    # print("REGISTERS: ", registers)
    # print("PROGRAM: ", program)
    # print("OPCODE: ", opcode)
    # print("OPERAND: ", operand)
    # print("COMBO: ", combo)
    # print("POINTER: ", pointer)
    # print("CALLING ",opcode2func[opcode])
    pointer, registers, out = opcode2func[opcode](opcode, operand, combo, pointer, registers, out)
    # print("-----------------AFTER---------------------")
    # print("REGISTERS: ", registers)
    # print("PROGRAM: ", program)
    # print("OPCODE: ", opcode)
    # print("OPERAND: ", operand)
    # print("POINTER: ", pointer)
    # print('============================================')
  #   if return_history:
  #     history.append((copy.deepcopy(pointer), copy.deepcopy(registers), copy.deepcopy(out)))
  # if return_history:
  #   return out, history
  else:
    return out 


def get_output(registers, program):
  return ','.join(map(str,execute(registers, program)))


def calculate_A_register(registers, program):
  def calculate_A_register_(registers, program, cur_num="", num_to_check=1):
    # TODO: BASICALLY NEED TO DO THIS BUT MAKE IT BE A DFS (We may see two binary_0_to_8s that satisfy the current sequence so we need to check them all)
    out = []
    binary_0_to_8 = ["{:b}".format(i).zfill(3) for i in range(0,8)]
    # print(binary_0_to_8)
    # print(program[-num_to_check:])
    # print("CUR NUM: ", cur_num)
    potential_solutions = []
    for b in binary_0_to_8:
      cur_registers = copy.deepcopy(registers)
      cur_program = copy.deepcopy(program)
      cur_registers['A'] = int(cur_num+b, 2)
      out = execute(cur_registers, cur_program)
      if out == program[-num_to_check:]:
          if out == program:
            potential_solutions.append(cur_num+b)
          else:
            potential_solutions.append(calculate_A_register_(registers, program, cur_num + b, num_to_check+1))
    # print("FLOOP", potential_solutions, num_to_check)
    out_num = b if len(potential_solutions) == 0 else max(potential_solutions, key=len)
    # print("OUT NUM: ", out_num)
    return out_num
  return int(calculate_A_register_(registers, program), 2)

3,6,7,0,5,7,3,1,4


In [17]:
# def diff(a):
#   return [a[i]-a[i-1] for i in range(1, len(a))]



# def calculate_A_register(registers, program, start_num=0, n_to_increase=1, n_to_check=1, n_sols=10):
#   cur_registers = copy.deepcopy(registers)
#   cur_program = copy.deepcopy(program)
#   out = [-1] * len(program)
#   cnt = int(start_num / 2) if start_num/2 ==int(start_num/2) else start_num
#   sols = []
#   outs = []
#   first = True
#   loopcnt = 0
#   while len(sols) < n_sols:
#     cur_registers, cur_program = copy.deepcopy(formatted_input)
#     cur_registers['A'] = cnt
#     out = execute(cur_registers, cur_program, return_history=False)
#     if first:
#       print("PROGRAM: ", cur_program[:n_to_check])
#       first = False
#     if out[:n_to_check] == cur_program[:n_to_check]:
#       print("DOOT")
#       sols.append(cnt)
#       outs.append(out)
#     cnt += n_to_increase
#     loopcnt += 1
#     if loopcnt%10000==0:
#       print(cnt)
#       print(cur_program[:n_to_check])
#       print(out)
#   # print("SD:", sols, diff(sols))
#   min_diff = min(diff(sols))
#   print(diff(sols))
#   min_sol = min(sols)
#   # print(n_to_check, ':', sols, min_sol, min_diff)
#   print(n_to_check, min_sol, min_diff)
#   # assert len(set([sol%i for sol in sols])) == 1
#   if n_to_check == len(program):
#     return min_sol
#   else:
#     return calculate_A_register(registers, program, min_sol, min_diff, n_to_check+1, n_sols)

In [18]:
# calculate_A_register(*formatted_input, n_to_check=4, n_sols=15)

In [222]:
# sss = [31157, 1079733, 5405109, 13793717, 1892841909, 1892841909, 1892841909, 12630260149]
# prev_maxi = 1
# for poo in range(len(sss)-1):
#   s = [sss[poo], sss[poo+1]]
#   print(s)
#   for i in range(1, s[0]+1, prev_maxi):
#     if len(set(map(lambda x: x%i, s)))==1:
#       maxi = i
#       maxrem = s[0]%i
#   print(maxi, maxrem)
#   prev_maxi = maxi

[31157, 1079733]
16384 14773
[1079733, 5405109]
1 0
[5405109, 13793717]
4194304 1210805
[13793717, 1892841909]
1 0
[1892841909, 1892841909]


KeyboardInterrupt: 

In [19]:
# calculate_A_register(*formatted_input, n_to_check=13, n_sols=12, start_num=12630260149, n_to_increase=4294967296)

In [20]:

# out = [-1] * len(formatted_input[1])
# cnt = 31157
# sols = []
# outs = []
# histories = []
# first = True
# while len(sols) < 12:
#   cur_registers, cur_program = copy.deepcopy(formatted_input)
#   cur_registers['A'] = cnt
#   out, history = execute(cur_registers, cur_program, return_history=True)
#   if first:
#     print("PROGRAM: ", cur_program[:4])
#     first = False
#   if out[:4] == cur_program[:4]:
#     print("DOO")
#     sols.append(cnt)
#     outs.append(out)
#     histories.append(history)
#   cnt += 128
#   # print("SET A TO: ", cnt)
# for i in range(len(sols)):
#   print("\n\n")
#   print(sols[i], ':', outs[i])
#   print(histories[i][:6])

In [21]:
# print(sols, outs)
# i = 0
# min_diff = np.inf
# max_same_mod = 1
# for i in range(1, sols[0]+1):
#   if len(set([sol%i for sol in sols])) == 1 and i > max_same_mod:
#     max_same_mod = i
#   diffs = np.diff(sols)[1:]
#   if min(diffs) < min_diff:
#     min_diff = min(diffs)
# print('\n')
# print(max_same_mod, sols[0]%i, diffs)
# print("MIN SOL", min(sols))
# print("MIN DIFF: ", min_diff)

In [23]:
# 3968/128

In [24]:
# [14+(64*i) for i in range(20)]

In [25]:
# 7^1

6

In [ ]:
# 52699261352180149

In [ ]:
# 281474976710656

In [26]:
# n = 281474976710655
# n = 281474976710655 - 246_290_604_621_823
# print(n)
# print('----')
# for i in range(16):
#   n = int(n/8)
#   print(n)

In [27]:
# n1 = 281474976710655
# n2 = 281474976710655 - 246_290_604_621_823

# from tqdm import tqdm
# for i in tqdm(range(n2, n1+1)):
#   10000 - 10000

In [28]:
# n2

In [495]:
# 281474976710655
# 35184372088832

# 35195454519733



4194303.999999985

In [29]:
# lowest = 1079733
# difference = 131072
# cnt = 268430000
# start_cnt = cnt
# while lowest + (difference*cnt) <= n1:
#   if cnt % 100000 == 0:
#     print(lowest + (difference*cnt))
#   if lowest + (difference*cnt) >= n2:
#     cur_registers = copy.deepcopy(formatted_input[0])
#     cur_program = copy.deepcopy(formatted_input[1])
#     cur_registers['A'] = lowest + (difference*cnt)
#     out = execute(cur_registers, cur_program, return_history = False)
#     assert(len(out) == len(cur_program))
#     # print(out)
#     if out == cur_program:
#       print("HOWOWOWOW, ", lowest + (difference*cnt))
#       break
#   elif cnt == start_cnt:
#     print("LOWER!!")
#   cnt += 1

In [30]:
# for i in tqdm(range(int((n1 - n2) / 128))):
#   1000 - 1000

In [ ]:
# cur_registers = copy.deepcopy(formatted_input[0])
# # cur_registers['A'] = 14 + 64 + 254

# cur_registers['A'] = 164278496489149
# # print(int(cur_registers['A']/8))
# # print(cur_registers['A']%8)
# cur_program = copy.deepcopy(formatted_input[1])
# out, history = execute(cur_registers, cur_program, return_history = True)
# print(out)
# if out == [3,0]:
#   print(out)
#   for h in history:
#     print(h)

In [33]:
# n = 298
# print("{0:b}".format(n))
# print(int(n/8))
# print(n%8)

In [34]:
# def calculate_A_register(registers, program):
#   mod_to_num = {}
#   for i in range(8):
#     cur_registers = copy.deepcopy(registers)
#     cur_program = copy.deepcopy(program)
#     cur_registers['A'] = i
#     mod_to_num[i] = execute(cur_registers, cur_program)[0]

#   a_register = mod_to_num[program[-1]]
#   for i, num in enumerate(reversed(program[:-1])):
#     a_register = (a_register*8)+mod_to_num[num]
#   return a_register

# a_register = calculate_A_register(*formatted_input)

# registers = copy.deepcopy(formatted_input[0])
# program = copy.deepcopy(formatted_input[1])
# registers['A'] = a_register
# execute(registers, program)

[0, 4, 7, 2, 5, 4, 0, 4, 4, 2, 4, 4, 7, 7, 3, 0]

In [361]:
# len("{0:b}".format(1892841909))%3/

1

In [244]:
# 4, 2, 5, 5, 0, 3, 3, 0]

39

In [357]:
# int("100101", 2)

37

In [346]:
# print("{0:b}".format(4))
# print("{0:b}".format(39))
# print("{0:b}".format(298))
# print("{0:b}".format(2390))

100
100111
100101010
100101010110


In [41]:
print(get_output(*copy.deepcopy(formatted_input)))
print(calculate_A_register(*formatted_input))

3,6,7,0,5,7,3,1,4
164278496489149
